In [1]:
import matplotlib

matplotlib.use('Agg')
import train
import dataset as ds
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from lstm_crf import EntityLSTM
import utils
import os
import conll2brat
import glob
import codecs
import shutil
import time
import copy
import evaluate
import random
import pickle
import brat2conll
import numpy as np
import utils_nlp
import distutils.util as distutils_util
import configparser
from pprint import pprint

parameters = {'pretrained_model_folder':'../model',
                      'dataset_text_folder':'../../../ML_EntityData/data/en',
                      'character_embedding_dimension':25,
                      'character_lstm_hidden_state_dimension':25,
                      'check_for_digits_replaced_with_zeros':True,
                      'check_for_lowercase':True,
                      'debug':False,
                      'dropout_rate':0.5,
                      'experiment_name':'test',
                      'freeze_token_embeddings':False,
                      'gradient_clipping_value':5.0,
                      'learning_rate':0.005,
                      'load_only_pretrained_token_embeddings':False,
                      'load_all_pretrained_token_embeddings':False,
                      'main_evaluation_mode':'conll',
                      'maximum_number_of_epochs':3,
                      'number_of_cpu_threads':8,
                      'number_of_gpus':0,
                      'optimizer':'sgd',
                      'output_folder':'../../../ML_EntityData/output',
                      'patience':10,
                      'plot_format':'pdf',
                      'reload_character_embeddings':True,
                      'reload_character_lstm':True,
                      'reload_crf':True,
                      'reload_feedforward':True,
                      'reload_token_embeddings':True,
                      'reload_token_lstm':True,
                      'remap_unknown_tokens_to_unk':True,
                      'spacylanguage':'en',
                      'tagging_format':'bioes',
                      'token_embedding_dimension':100,
                      'token_lstm_hidden_state_dimension':100,
                      'token_pretrained_embedding_filepath':'../../../ML_EntityData/embedding/en/glove.6B.100d.txt',
                      'tokenizer':'spacy',
                      'train_model':True,
                      'use_character_lstm':True,
                      'use_crf':True,
                      'use_pretrained_model':False,
                      'verbose':False}



### Load Dataset

In [1]:

# Load dataset
dataset_filepaths, dataset_brat_folders = utils.get_valid_dataset_filepaths(parameters)
dataset = ds.Dataset(verbose=False, debug=False)
token_to_vector = dataset.load_dataset(dataset_filepaths, parameters)


NameError: name 'utils' is not defined

### Create Model

In [3]:

# Create model lstm+crf
session_conf = tf.ConfigProto(
            intra_op_parallelism_threads=parameters['number_of_cpu_threads'],
            inter_op_parallelism_threads=parameters['number_of_cpu_threads'],
            device_count={'CPU': 1, 'GPU': parameters['number_of_gpus']},
            allow_soft_placement=True,
            # automatically choose an existing and supported device to run the operations in case the specified one doesn't exist
            log_device_placement=False
        )
sess = tf.Session(config=session_conf)

with sess.as_default():
    # Create model and initialize or load pretrained model
    ### Instantiate the model
    model = EntityLSTM(dataset=dataset, token_embedding_dimension=parameters['token_embedding_dimension'],
                       character_lstm_hidden_state_dimension=parameters['character_lstm_hidden_state_dimension'],
                       token_lstm_hidden_state_dimension=parameters['token_lstm_hidden_state_dimension'],
                       character_embedding_dimension=parameters['character_embedding_dimension'],
                       use_crf=parameters['use_crf'],
                       use_character_lstm=parameters['use_character_lstm'],
                       gradient_clipping_value=parameters['gradient_clipping_value'],
                       learning_rate=parameters['learning_rate'],
                       freeze_token_embeddings=parameters['freeze_token_embeddings'],
                       optimizer=parameters['optimizer'],
                       maximum_number_of_epochs=parameters['maximum_number_of_epochs'])

sess.run(tf.global_variables_initializer())

d:\programs\python\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### Initial Model

In [4]:
# Load embedding
model.load_pretrained_token_embeddings(sess, dataset,embedding_filepath=parameters['token_pretrained_embedding_filepath'],
                                                       check_lowercase= parameters['check_for_lowercase'],check_digits=parameters['check_for_digits_replaced_with_zeros'],
                                                       token_to_vector=token_to_vector)
# Initial params_train
transition_params_trained = np.random.rand(len(dataset.unique_labels) + 2,len(dataset.unique_labels) + 2)

del token_to_vector

Load token embeddings... done (0.38 seconds)
number_of_token_original_case_found: 14618
number_of_token_lowercase_found: 11723
number_of_token_digits_replaced_with_zeros_found: 119
number_of_token_lowercase_and_digits_replaced_with_zeros_found: 16
number_of_loaded_word_vectors: 26476
dataset.vocabulary_size: 28984


### Restore Model

In [4]:

transition_params_trained = model.restore_from_pretrained_model(dataset, sess , model_pathfile=os.path.join(parameters['pretrained_model_folder'],'model.ckpt'),
                                                                                     dataset_pathfile=(parameters['pretrained_model_folder']+'/dataset.pickle'),
                                                                                     embedding_filepath= parameters['token_pretrained_embedding_filepath'],
                                                                                     character_dimension = parameters['character_embedding_dimension'],
                                                                                     token_dimension=parameters['token_embedding_dimension'],token_to_vector=token_to_vector)
del token_to_vector

INFO:tensorflow:Restoring parameters from ../model\model.ckpt


In [6]:
stats_graph_folder, experiment_timestamp = utils.create_stats_graph_folder(parameters)

        # Initialize and save execution details
start_time = time.time()
results = {}
results['epoch'] = {}
results['execution_details'] = {}
results['execution_details']['train_start'] = start_time
results['execution_details']['time_stamp'] = experiment_timestamp
results['execution_details']['early_stop'] = False
results['execution_details']['keyboard_interrupt'] = False
results['execution_details']['num_epochs'] = 0
results['model_options'] = copy.copy(parameters)

model_folder = os.path.join(stats_graph_folder, 'model')
utils.create_folder_if_not_exists(model_folder)

pickle.dump(dataset, open(os.path.join(model_folder, 'dataset.pickle'), 'wb'))

# tensorboard_log_folder = os.path.join(stats_graph_folder, 'tensorboard_logs')
# utils.create_folder_if_not_exists(tensorboard_log_folder)
# tensorboard_log_folders = {}
# for dataset_type in dataset_filepaths.keys():
#     tensorboard_log_folders[dataset_type] = os.path.join(stats_graph_folder, 'tensorboard_logs', dataset_type)
#     utils.create_folder_if_not_exists(tensorboard_log_folders[dataset_type])

# Instantiate the writers for TensorBoard
# writers = {}
# for dataset_type in dataset_filepaths.keys():
#     writers[dataset_type] = tf.summary.FileWriter(tensorboard_log_folders[dataset_type], graph=sess.graph)
# embedding_writer = tf.summary.FileWriter(
#     model_folder)  # embedding_writer has to write in model_folder, otherwise TensorBoard won't be able to view embeddings

# embeddings_projector_config = projector.ProjectorConfig()
# tensorboard_token_embeddings = embeddings_projector_config.embeddings.add()
# tensorboard_token_embeddings.tensor_name = model.token_embedding_weights.name
# token_list_file_path = os.path.join(model_folder, 'tensorboard_metadata_tokens.tsv')
# tensorboard_token_embeddings.metadata_path = os.path.relpath(token_list_file_path, '..')

# tensorboard_character_embeddings = embeddings_projector_config.embeddings.add()
# tensorboard_character_embeddings.tensor_name = model.character_embedding_weights.name
# character_list_file_path = os.path.join(model_folder, 'tensorboard_metadata_characters.tsv')
# tensorboard_character_embeddings.metadata_path = os.path.relpath(character_list_file_path, '..')

# projector.visualize_embeddings(embedding_writer, embeddings_projector_config)

# Write metadata for TensorBoard embeddings
# token_list_file = codecs.open(token_list_file_path, 'w', 'UTF-8')
# for token_index in range(dataset.vocabulary_size):
#     token_list_file.write('{0}\n'.format(dataset.index_to_token[token_index]))
# token_list_file.close()

# character_list_file = codecs.open(character_list_file_path, 'w', 'UTF-8')
# for character_index in range(dataset.alphabet_size):
#     if character_index == dataset.PADDING_CHARACTER_INDEX:
#         character_list_file.write('PADDING\n')
#     else:
#         character_list_file.write('{0}\n'.format(dataset.index_to_character[character_index]))
# character_list_file.close()



In [9]:
bad_counter = 0  # number of epochs with no improvement on the validation test in terms of F1-score
previous_best_valid_f1_score = 0
epoch_number = -1
try:
    while True:

        step = 0
        epoch_number += 1
        print('\nStarting epoch {0}'.format(epoch_number))

        epoch_start_time = time.time()

        if epoch_number != 0:
            # Train model: loop over all sequences of training set with shuffling
            sequence_numbers = list(range(len(dataset.token_indices['train'])))
            random.shuffle(sequence_numbers)
            for sequence_number in sequence_numbers:
                transition_params_trained = train.train_step(sess, dataset, sequence_number, model, parameters['dropout_rate'])
                step += 1
                if step % 10 == 0:
                    print('Training {0:.2f}% done'.format(step / len(sequence_numbers) * 100), end='\r', flush=True)

        epoch_elapsed_training_time = time.time() - epoch_start_time
        print('Training completed in {0:.2f} seconds'.format(epoch_elapsed_training_time), flush=True)

        y_pred, y_true, output_filepaths = train.predict_labels_lite(sess=sess,model= model,transition_params_trained= transition_params_trained,
                                                                         dataset=dataset,epoch_number= epoch_number,
                                                                        stats_graph_folder= stats_graph_folder,dataset_filepaths= dataset_filepaths,
                                                                        tagging_format= parameters['tagging_format'], main_evaluation_mode=parameters['main_evaluation_mode'],use_crf=parameters['use_crf'])

        # # Evaluate model: save and plot results
        # evaluate.evaluate_model(results, dataset, y_pred, y_true, stats_graph_folder, epoch_number,
        #                                 epoch_start_time, output_filepaths, parameters)
        #
        # if parameters['use_pretrained_model'] and not parameters['train_model']:
        #     conll2brat.output_brat(output_filepaths, dataset_brat_folders, stats_graph_folder)
        #     break
        #
        # # Save model
        model.saver.save(sess, os.path.join(model_folder, 'model_{0:05d}.ckpt'.format(epoch_number)))
        #
        # # Save TensorBoard logs
        # summary = sess.run(model.summary_op, feed_dict=None)
        # writers['train'].add_summary(summary, epoch_number)
        # writers['train'].flush()
        # utils.copytree(writers['train'].get_logdir(), model_folder)
        #
        # # Early stop
        # valid_f1_score = results['epoch'][epoch_number][0]['valid']['f1_score']['micro']
        # if valid_f1_score > previous_best_valid_f1_score:
        #     bad_counter = 0
        #     previous_best_valid_f1_score = valid_f1_score
        #     conll2brat.output_brat(output_filepaths, dataset_brat_folders, stats_graph_folder,
        #                                       overwrite=True)
        #     transition_params_trained = transition_params_trained
        # else:
        #     bad_counter += 1
        # print("The last {0} epochs have not shown improvements on the validation set.".format(bad_counter))
        #
        # if bad_counter >= parameters['patience']:
        #     print('Early Stop!')
        #     results['execution_details']['early_stop'] = True
        #     break

        if epoch_number >= parameters['maximum_number_of_epochs']: break


except KeyboardInterrupt:
    results['execution_details']['keyboard_interrupt'] = True
    print('Training interrupted')

print('Finishing the experiment')
end_time = time.time()
print(end_time-start_time)
# results['execution_details']['train_duration'] = end_time - start_time
# results['execution_details']['train_end'] = end_time
# evaluate.save_results(results, stats_graph_folder)
# for dataset_type in dataset_filepaths.keys():
#     writers[dataset_type].close()



Starting epoch 0
Training completed in 0.00 seconds
Evaluate model on the train set
             precision    recall  f1-score   support

      B-LOC     0.9335    0.8905    0.9115      1041
     B-MISC     0.9122    0.7751    0.8381       858
      B-ORG     0.9359    0.8757    0.9048      2485
      B-PER     0.9466    0.9897    0.9677      4284
      E-LOC     0.9353    0.8886    0.9113      1041
     E-MISC     0.9261    0.7890    0.8521       858
      E-ORG     0.9401    0.8777    0.9078      2485
      E-PER     0.9517    0.9939    0.9724      4284
      I-LOC     0.8049    0.5690    0.6667       116
     I-MISC     0.8829    0.6599    0.7553       297
      I-ORG     0.7845    0.9286    0.8505      1219
      I-PER     0.8866    0.8975    0.8921       244
          O     0.0000    0.0000    0.0000       490
      S-LOC     0.9651    0.9647    0.9649      6099
     S-MISC     0.9188    0.9039    0.9113      2580
      S-ORG     0.9553    0.8918    0.9225      3836
      S-PER  


Evaluate model on the test set
             precision    recall  f1-score   support

      B-LOC     0.7795    0.8534    0.8148       232
     B-MISC     0.6188    0.6328    0.6257       177
      B-ORG     0.8453    0.8117    0.8282       579
      B-PER     0.9729    0.9899    0.9813      1086
      E-LOC     0.7747    0.8448    0.8082       232
     E-MISC     0.6464    0.6610    0.6536       177
      E-ORG     0.8559    0.8204    0.8377       579
      E-PER     0.9737    0.9880    0.9808      1086
      I-LOC     0.5500    0.4400    0.4889        25
     I-MISC     0.6154    0.4103    0.4923        39
      I-ORG     0.7753    0.8086    0.7916       256
      I-PER     0.9286    0.9286    0.9286        70
          O     0.0000    0.0000    0.0000       235
      S-LOC     0.9278    0.9311    0.9294      1436
     S-MISC     0.8659    0.8114    0.8378       525
      S-ORG     0.8941    0.8817    0.8879      1082
      S-PER     0.9457    0.8531    0.8970       531

avg / total 

In [7]:
prediction_count=0
def predict( text):
        

#         if prediction_count == 1:
        parameters['dataset_text_folder'] = os.path.join('..', 'data', 'temp')
        stats_graph_folder, _ = utils.create_stats_graph_folder(parameters)

        # Update the deploy folder, file, and dataset
        dataset_type = 'deploy'
        ### Delete all deployment data
        for filepath in glob.glob(os.path.join(parameters['dataset_text_folder'], '{0}*'.format(dataset_type))):
            if os.path.isdir(filepath):
                shutil.rmtree(filepath)
            else:
                os.remove(filepath)
        ### Create brat folder and file
        dataset_brat_deploy_folder = os.path.join(parameters['dataset_text_folder'], dataset_type)
        utils.create_folder_if_not_exists(dataset_brat_deploy_folder)
        dataset_brat_deploy_filepath = os.path.join(dataset_brat_deploy_folder, 'temp_{0}.txt'.format(
            str(prediction_count).zfill(5)))  # self._get_dataset_brat_deploy_filepath(dataset_brat_deploy_folder)
        with codecs.open(dataset_brat_deploy_filepath, 'w', 'UTF-8') as f:
            f.write(text)
        ### Update deploy filepaths
        dataset_filepaths, dataset_brat_folders = utils.get_valid_dataset_filepaths(parameters,
                                                                                    dataset_types=[dataset_type])
        dataset_filepaths.update(dataset_filepaths)
        dataset_brat_folders.update(dataset_brat_folders)
        ### Update the dataset for the new deploy set
        dataset.update_dataset(dataset_filepaths, [dataset_type])

        # Predict labels and output brat
        output_filepaths = {}
        prediction_output = train.prediction_step(sess, dataset, dataset_type, model,
                                                  transition_params_trained, stats_graph_folder,
                                                  prediction_count, dataset_filepaths,parameters['tagging_format'],
                                                 parameters['main_evaluation_mode'])
        _, _, output_filepaths[dataset_type] = prediction_output
        conll2brat.output_brat(output_filepaths, dataset_brat_folders, stats_graph_folder, overwrite=True)

        # Print and output result
        text_filepath = os.path.join(stats_graph_folder, 'brat', 'deploy',
                                     os.path.basename(dataset_brat_deploy_filepath))
        annotation_filepath = os.path.join(stats_graph_folder, 'brat', 'deploy', '{0}.ann'.format(
            utils.get_basename_without_extension(dataset_brat_deploy_filepath)))
        text2, entities = brat2conll.get_entities_from_brat(text_filepath, annotation_filepath, verbose=True)
        assert (text == text2)
        return entities


In [8]:
predict('my name is Ngoc Linh')

Formatting deploy set from BRAT to CONLL... Done.
Converting CONLL from BIO to BIOES format... Done.
Predict labels for the deploy set
Formatting 000_deploy set from CONLL to BRAT... Done.

text:
my name is Ngoc Linh

entity: {'id': 'T1', 'type': 'PER', 'start': 11, 'end': 20, 'text': 'Ngoc Linh'}





[{'end': 20, 'id': 'T1', 'start': 11, 'text': 'Ngoc Linh', 'type': 'PER'}]

In [10]:
predict("Vietnam is a country in Southeast Asia. The long-form name of the country is the Socialist Republic of Vietnam. The neighboring countries of Vietnam are China, Laos, and Cambodia. Vietnam is one of five remaining countries that believe in communism. The capital of Vietnam is Hanoi. The biggest city is Ho Chi Minh City (formerly Saigon). There are about 94,444,200 people living in Vietnam.After the Japanese occupation in the 1940s, the Vietnamese fought French colonial rule during the First Indochina War between the Viet Minh and the French in 2 September 1945. Hồ_Chí_Minh declared Vietnam's independence from France under the new name of the Democratic Republic of Vietnam, but was fought by capitalist forces. In 1954, the Vietnamese declared victory in Dien Bien Phu which took place between March and May 1954 and culminated in a major French defeat. Then Vietnam was divided into two political states, North Vietnam (officially the Democratic Republic of Vietnam) and South Vietnam (officially the Republic of Vietnam). Conflicts between the two sides intensified in the so-called Vietnam War with strong influence from the US in South Vietnam. The war ended in 1975 with a North Vietnamese victory.Vietnam was then united under a communist government. In 1986, the government launched a number of economic and political reforms that began Vietnam's path to integration into the world economy.By 2000, it had established diplomatic relations with all nations. Since 2000, Vietnam's economic growth has been among the highest in the world , and in 2011 it had the highest global growth generator index among 11 major economies. Its successful economic reforms resulted in its becoming a member of the World Trade Organization in 2007. It is also a member of economic cooperation between Asia and the Pacific and the International de la Francophonie Organization.")

Formatting deploy set from BRAT to CONLL... Done.
Converting CONLL from BIO to BIOES format... Done.
Predict labels for the deploy set
Formatting 000_deploy set from CONLL to BRAT... Done.

text:
Vietnam is a country in Southeast Asia. The long-form name of the country is the Socialist Republic of Vietnam. The neighboring countries of Vietnam are China, Laos, and Cambodia. Vietnam is one of five remaining countries that believe in communism. The capital of Vietnam is Hanoi. The biggest city is Ho Chi Minh City (formerly Saigon). There are about 94,444,200 people living in Vietnam.After the Japanese occupation in the 1940s, the Vietnamese fought French colonial rule during the First Indochina War between the Viet Minh and the French in 2 September 1945. Hồ_Chí_Minh declared Vietnam's independence from France under the new name of the Democratic Republic of Vietnam, but was fought by capitalist forces. In 1954, the Vietnamese declared victory in Dien Bien Phu which took place between Mar

[{'end': 7, 'id': 'T1', 'start': 0, 'text': 'Vietnam', 'type': 'LOC'},
 {'end': 38, 'id': 'T2', 'start': 24, 'text': 'Southeast Asia', 'type': 'LOC'},
 {'end': 110,
  'id': 'T3',
  'start': 81,
  'text': 'Socialist Republic of Vietnam',
  'type': 'ORG'},
 {'end': 148, 'id': 'T4', 'start': 141, 'text': 'Vietnam', 'type': 'LOC'},
 {'end': 158, 'id': 'T5', 'start': 153, 'text': 'China', 'type': 'LOC'},
 {'end': 164, 'id': 'T6', 'start': 160, 'text': 'Laos', 'type': 'LOC'},
 {'end': 178, 'id': 'T7', 'start': 170, 'text': 'Cambodia', 'type': 'LOC'},
 {'end': 187, 'id': 'T8', 'start': 180, 'text': 'Vietnam', 'type': 'LOC'},
 {'end': 272, 'id': 'T9', 'start': 265, 'text': 'Vietnam', 'type': 'LOC'},
 {'end': 281, 'id': 'T10', 'start': 276, 'text': 'Hanoi', 'type': 'LOC'},
 {'end': 319,
  'id': 'T11',
  'start': 303,
  'text': 'Ho Chi Minh City',
  'type': 'ORG'},
 {'end': 336, 'id': 'T12', 'start': 330, 'text': 'Saigon', 'type': 'LOC'},
 {'end': 390, 'id': 'T13', 'start': 383, 'text': 'Vietnam